In [1]:
import pandas as pd
import numpy as np

In [2]:
mem_data = pd.read_csv('kaggle_data/mem_data.csv')
mem_tr = pd.read_csv('kaggle_data/mem_transaction.csv')
s_info = pd.read_csv('kaggle_data/store_info.csv')

In [3]:
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10764 entries, 0 to 10763
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEM_ID       10764 non-null  int64  
 1   M_STORE_ID   10764 non-null  int64  
 2   GENDER       10764 non-null  object 
 3   BIRTH_DT     5997 non-null   object 
 4   BIRTH_SL     10764 non-null  object 
 5   ZIP_CD       10764 non-null  object 
 6   RGST_DT      10764 non-null  object 
 7   VISIT_CNT    10764 non-null  float64
 8   SALES_AMT    10764 non-null  float64
 9   LAST_VST_DT  10764 non-null  object 
 10  USABLE_PNT   10764 non-null  float64
 11  USED_PNT     10764 non-null  float64
 12  ACC_PNT      10764 non-null  float64
 13  USABLE_INIT  10764 non-null  float64
 14  SMS          10764 non-null  object 
dtypes: float64(6), int64(2), object(7)
memory usage: 1.2+ MB


In [4]:
mem_tr.head()

,STORE_ID,SELL_DT,MEMP_STY,MEM_ID,MEMP_DT,SELL_AMT,MEMP_TP
0,125,20060227,O,1225434,2006-02-27 13:57:44.750000000,15500,A
1,125,20060303,O,1181938,2006-03-03 15:43:56.987000000,22600,A
2,125,20060322,O,6093,2006-03-22 18:08:45.563000000,11000,A
3,125,20060403,O,1398202,2006-04-03 13:36:27.040000000,14400,A
4,125,20060418,O,1225434,2006-04-18 16:04:39.583000000,6600,A


In [5]:
mem_data.GENDER.value_counts()

F          3767
M          3767
UNKNOWN    3230
Name: GENDER, dtype: int64

In [6]:
mem_data.SMS.value_counts()

Y    10575
N      189
Name: SMS, dtype: int64

In [7]:
# SMS 수신동의 정수 처리하기
mem_data.SMS = (mem_data.SMS=='Y').astype(int)
mem_data.SMS.value_counts()

1    10575
0      189
Name: SMS, dtype: int64

In [8]:
# 양/음력(BIRTH_SL) 정수 처리하기
mem_data.BIRTH_SL = (mem_data.BIRTH_SL=='Y').astype(int)
mem_data.BIRTH_SL.value_counts()

0    10764
Name: BIRTH_SL, dtype: int64

In [9]:
# 구매 합계(SALES_AMT) 로그 처리 하기
# 참고 : 로그처리에는 음수가 들어갈 수 없음 / 0이 있을 경우 1을 더하고 처리
f = mem_data.SALES_AMT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.SALES_AMT = f

In [10]:
# 최근 방문 일자(LAST_VST_DT)로부터 경과일 구하기
f = pd.to_datetime(mem_data.LAST_VST_DT) # 날짜가 Object 이므로 숫자 처리
f = (pd.to_datetime('2007-12-13') - f).dt.days #방문이 얼마나 오래되었는지 ,time 등으로 알고 싶은 것으로 바꿀 수 있다.
mem_data['E_DAY'] = f
mem_data.E_DAY.head()

0    327
1     55
2    196
3     57
4    262
Name: E_DAY, dtype: int64

In [11]:
f = pd.to_datetime(mem_data.LAST_VST_DT)
f = (pd.to_datetime('2007-12-13') - f)

In [12]:
f[f.isna()]

Series([], Name: LAST_VST_DT, dtype: timedelta64[ns])

In [13]:
mem_data.LAST_VST_DT[1612]

'2006-06-02 11:31:20.640000000'

In [14]:
f.isna().sum()

0

In [15]:
mem_data = mem_data.merge(mem_tr, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [16]:
d_col = ['BIRTH_SL','M_STORE_ID','BIRTH_DT','ZIP_CD','RGST_DT','LAST_VST_DT', 'MEMP_STY', 'MEMP_DT', 'MEMP_TP']
mem_data = mem_data.drop(d_col, axis=1)
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60049 entries, 0 to 60048
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEM_ID       60049 non-null  int64  
 1   GENDER       60049 non-null  object 
 2   VISIT_CNT    60049 non-null  float64
 3   SALES_AMT    60049 non-null  float64
 4   USABLE_PNT   60049 non-null  float64
 5   USED_PNT     60049 non-null  float64
 6   ACC_PNT      60049 non-null  float64
 7   USABLE_INIT  60049 non-null  float64
 8   SMS          60049 non-null  int32  
 9   E_DAY        60049 non-null  int64  
 10  STORE_ID     60049 non-null  int64  
 11  SELL_DT      60049 non-null  int64  
 12  SELL_AMT     60049 non-null  int64  
dtypes: float64(6), int32(1), int64(5), object(1)
memory usage: 6.2+ MB


In [17]:
np.random.seed(123)

tr = mem_data.GENDER!='UNKNOWN'
train = mem_data[tr]
train.GENDER = (train.GENDER=='M').astype(int)

C:\Users\Big4s\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [18]:
te = mem_data.GENDER=='UNKNOWN'
test = mem_data[te].sort_values('MEM_ID')
test.head(3)

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,STORE_ID,SELL_DT,SELL_AMT
50817,1054,UNKNOWN,5.0,12.358369,29158.0,0.0,29158.0,846.0,0,57,1421,20060315,15400
50816,1054,UNKNOWN,5.0,12.358369,29158.0,0.0,29158.0,846.0,0,57,1274,20071016,17600
50815,1054,UNKNOWN,5.0,12.358369,29158.0,0.0,29158.0,846.0,0,57,1274,20061026,14800


In [19]:
train.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,STORE_ID,SELL_DT,SELL_AMT
0,1134945,1,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060315,10000
1,1134945,1,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060404,22300
2,1134945,1,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060601,32000
3,1134945,1,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20061030,3000
4,1134945,1,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20061030,15300


In [20]:
mem_data.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,E_DAY,STORE_ID,SELL_DT,SELL_AMT
0,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060315,10000
1,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060404,22300
2,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20060601,32000
3,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20061030,3000
4,1134945,M,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,1084,20061030,15300


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import glob

In [22]:
kfold = StratifiedKFold(n_splits=5) # 하이퍼 파라미터 지정
n_it = 12

In [23]:
t_final = test[['MEM_ID', 'GENDER']]
t_final.head()

,MEM_ID,GENDER
50817,1054,UNKNOWN
50816,1054,UNKNOWN
50815,1054,UNKNOWN
50814,1054,UNKNOWN
50813,1054,UNKNOWN


In [24]:
test = test.drop(['GENDER','MEM_ID'], axis=1)
target = train.GENDER.values
train = train.drop(['GENDER','MEM_ID'], axis=1)

In [25]:
print(train.shape, test.shape)

(39569, 11) (20480, 11)


In [26]:
params = {'max_features':list(np.arange(1, train.shape[1])), 'bootstrap':[False], 'n_estimators': [50], 'criterion':['gini','entropy']}
model = RandomizedSearchCV(RandomForestClassifier(), param_distributions=params, n_iter=n_it, cv=kfold, scoring='roc_auc',n_jobs=-1, verbose=1)
print('MODELING.............................................................................')
model.fit(train, target)
print('========BEST_AUC_SCORE = ', model.best_score_)
model = model.best_estimator_
t_final.GENDER = model.predict_proba(test.values)[:,1]

MODELING.............................................................................
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.0min finished


========BEST_AUC_SCORE =  0.7338613334432844


In [27]:
t_final.head()

,MEM_ID,GENDER
50817,1054,0.58
50816,1054,0.68
50815,1054,0.66
50814,1054,0.54
50813,1054,0.54


In [28]:
t_final.to_csv('output_data.csv', index=False)
print('COMPLETE')

COMPLETE
